In [1]:
import pandas as pd

In [2]:
events = pd.read_csv('/Users/arturoreza/Documents/isac_2025/events_360_data/events_df_Liga_MX_2024_2025.csv')
matches = pd.read_csv('/Users/arturoreza/Documents/isac_2025/matches_data/season_matches_2024_2025.csv')

/var/folders/mm/5rp42wfn61b3g1r33d3fnfvc0000gn/T/ipykernel_14142/3827937734.py:1: DtypeWarning: Columns (1,2,5,6,7,8,9,10,11,12,13,14,15,16,17,18,20,21,22,23,24,25,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,51,52,57,66,67,68,70,71,72,73,74,75,76,77,78,80,81,82,83,85,88,90,91,92,93,94,95,100,107,108,109,110,111,112,113,117,118,119,120,121,122,123,126,127,129,130,131,133,135,140) have mixed types. Specify dtype option on import or set low_memory=False.
  events = pd.read_csv('/Users/arturoreza/Documents/isac_2025/events_360_data/events_df_Liga_MX_2024_2025.csv')


In [3]:
# Data copies
events_1 = events.copy()
matches_1 = matches.copy()

In [4]:
# Merging event with matches data
df = events.merge(
    matches[['match_id', 'home_team', 'away_team']],
    on='match_id',
    how='left'
)

In [5]:
# Prefer timestamp if present; else fall back to minute+second
if 'timestamp' in df.columns and df['timestamp'].notna().any():
    # StatsBomb-like "HH:MM:SS.mmm" or "MM:SS.mmm"
    df['ts'] = pd.to_timedelta(df['timestamp'])
else:
    df['ts'] = (pd.to_timedelta(df['minute'], unit='m')
                + pd.to_timedelta(df.get('second', 0), unit='s'))


In [6]:
# Order events within each match
sort_keys = ['match_id', 'period', 'ts']
if 'index' in df.columns:  # your file has an 'index' sequence – great tie-breaker
    sort_keys.append('index')
df = df.sort_values(sort_keys, kind='mergesort').reset_index(drop=True)

In [7]:
# Per event goal deltas from the home perspective
df['home_goal_evt'] = ((df['shot_outcome'] == 'Goal') & (df['team'] == df['home_team'])).astype(int)
df['away_goal_evt'] = ((df['shot_outcome'] == 'Goal') & (df['team'] == df['away_team'])).astype(int)

In [8]:
df['goal_delta'] = df['home_goal_evt'] - df['away_goal_evt']  # +1 if home scores, -1 if away scores, 0 otherwise

In [9]:
# Game state BEFORE the current event:
df['game_state'] = df.groupby('match_id')['goal_delta'].cumsum().shift(fill_value=0)

In [10]:
df = df.drop(columns=['ts', 'home_goal_evt', 'away_goal_evt', 'goal_delta'])

In [11]:
df.to_csv('events_df_Liga_MX_2024_2025.csv')